# Compute the Budyko precipitation recycling ratio

In [1]:
import os
import glob
import math
import numpy as np
import xarray as xr
import pickle

# Regridding
# import xesmf as xe

def find_dl(lat, lons):
    """
    Find average length of 1 grid cell - convert lat to radians, take cos +
    divide by Earth's circumference (m)
    """
    earth_circ = (40075*10**3)  # Earth's circumference in m
    lon_shape = len([lons])
    lat_rad = np.radians(abs(lat))
    dl = math.cos(lat_rad) * (earth_circ / lon_shape)  # length of pixel in m
    return dl

def lon180(ds):
    ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
    ds = ds.sortby(ds.lon)
    return ds

def get_pixel_size(lat, lon):
    if lat[0] > lat[-1]:
        temp_lat = lat[::-1]
    else:
        temp_lat = lat
    r = 6.371 * 1e6
    rad = (2 * math.pi / 360)  # (m)
    da = np.nan * np.zeros((len(temp_lat)))  # (m^2)

    for i in range(len(temp_lat) - 1):
        da[i] = (2 * math.pi * (1 / len(lon)) *
                 r ** 2 * (math.sin(rad * temp_lat[i + 1]) -
                           math.sin(rad * temp_lat[i])))

    # Check if top and bottom latitude are same
    if temp_lat[0] == -temp_lat[-1]:
        da[-1] = da[0]
    return da

In [2]:
data_path = "G:/My Drive/MPIM/data/"

In [3]:
import pickle

models = [ "ACCESS-ESM1-5", "CanESM5", "IPSL-CM6A-LR", "MPI-ESM1-2-LR", "UKESM1-0-LL", ]
pr_rec = []
for m,mm in enumerate(models): 
    with open(os.path.join(data_path+"budyko_pr_"+mm), "rb") as fp:   #Pickling
        pr_rec.append(pickle.load(fp)) 


In [30]:
np.nanmin((pr_rec[4]["Rr"].sel(time = slice("2070-01","2100-12")).mean(dim = "time")).values)


-36.290465228439494